In [2]:
pip install pandas numpy statsmodels arch scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.1/985.1 kB 9.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from arch import arch_model
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [14]:
df = pd.read_csv("Raw Data/IN_Cleaned.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1087 entries, 0 to 1086
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1087 non-null   object 
 1   PX_LAST      1087 non-null   float64
 2   PX_VOLUME_x  1087 non-null   int64  
 3   NIFVIX       1087 non-null   float64
 4   IN003M       1087 non-null   float64
 5   PX_BID       1087 non-null   float64
 6   PX_VOLUME_y  1087 non-null   int64  
 7   PX_ASK       1087 non-null   float64
 8   PX_SPREAD    1087 non-null   float64
dtypes: float64(6), int64(2), object(1)
memory usage: 76.6+ KB


In [16]:
def black_scholes_call(S, K, T, r, sigma):
    """
    Calculate the Black-Scholes option price.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T ) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    C = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return C

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')

df['Log_Returns'] = np.log(df['PX_LAST'] / df['PX_LAST'].shift(1))
df = df.dropna()  # Remove any NaN values resulting from shift operation

model = arch_model(df['Log_Returns'], vol='Garch', p=1, q=1, dist='normal', rescale=False)
model_fit = model.fit(disp="off")
df['GARCH_Volatility'] = model_fit.conditional_volatility
df['Sigma_Annualized'] = df['GARCH_Volatility'] * np.sqrt(252)
df['Daily_STIR'] = df['IN003M']/252

df['S'] = df['PX_LAST']              # Spot price (index price on that day)
df['K'] = df['PX_LAST']              # Set strike price equal to spot price (ATM option)
df['T'] = 1/252                      # Time to expiry (1 day in years)
df['r'] = df['Daily_STIR']               # Daily short-term interest rate (SOFR3M)
df['Sigma'] = df['GARCH_Volatility'] # Use annualized GARCH volatility as sigma

df['Option_Price'] = df.apply(
    lambda row: black_scholes_call(row['S'], row['K'], row['T'], row['r'], row['Sigma']), axis=1
    )

df_Model = (df[['Date', 'PX_LAST', 'GARCH_Volatility', 'Sigma_Annualized', 'NIFVIX', 'IN003M', 'Daily_STIR', 'Option_Price',]])
df_Model.info()
df_Model.to_csv("IN_Model.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1085 entries, 1084 to 0
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1085 non-null   datetime64[ns]
 1   PX_LAST           1085 non-null   float64       
 2   GARCH_Volatility  1085 non-null   float64       
 3   Sigma_Annualized  1085 non-null   float64       
 4   NIFVIX            1085 non-null   float64       
 5   IN003M            1085 non-null   float64       
 6   Daily_STIR        1085 non-null   float64       
 7   Option_Price      1085 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 76.3 KB
